<a href="https://colab.research.google.com/github/Maagnitude/wikimovieplot-nlp-model/blob/main/wikimovieplot-nlp-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **3η Εργασία** στο μάθημα **Μηχανική Μάθηση και Εφαρμογές**

# **Τμήμα Πληροφορικής και Τηλεματικής - Χαροκόπειο Πανεπιστήμιο**

# **Καζάζης Γεώργιος - it214124**

Στην παρούσα εργασία θα χρησιμοποιήσουμε μεθόδους επεξεργασίας φυσικής γλώσσας.

## **Modules**
Κάνουμε import τις απαραίτητες βιβλιοθήκες:
* **pandas** και **numpy** για την διαχείριση των δεδομένων μας.
* **re** για την επεξεργασία κειμένου (lowercase κτλ.)
* **tensorflow**, και από αυτήν το **keras** καθώς και τα **layers**, **losses**, **preprocessing** για την ανάπτυξη Νευρωνικών Δικτύων, και την εκπαίδευση τους.
* **TextVectorization** για να κάνουμε **vectorize** τα **Plot** και **Title** για την χρήση τους στην εκπαίδευση.
* Τα **warnings** για να τα φιλτράρουμε, ώστε να μην εμφανίζονται 

In [ ]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import losses
from keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import warnings
warnings.filterwarnings(action='ignore')

##**Dataset**
Περνάμε το link από το **public github repo** μου, για το **dataset** μας, σε μορφή **csv**, το οποίο θα δώσουμε στην συνάρτηση για το φόρτωμα των δεδομένων μας.

In [ ]:
url = "https://raw.githubusercontent.com/Maagnitude/wikimovieplot-nlp-model/main/dataset.csv"

## **Text preprocessing**
Με αυτή την συνάρτηση θα επεξεργαστούμε τα Title και Plot, ώστε να έχουν όλα πεζά γράμματα, να μην έχουν ειδικούς χαρακτήρες και σημεία στίξης.

In [ ]:
def preprocess_text(text):
    text = text.lower() # convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = re.sub(r'\s+', ' ', text)
    return text

## **Βασική συνάρτηση preprocessing**
Εδώ υλοποιείται το μεγαλύτερο κομμάτι του notebook. 

Αρχικά φορτώνεται το dataset, ανακατεύονται οι γραμμές του με την sample, χωρίζεται σε train, validate και test set (έχοντας υπολογίσει τα ποσοστά του καθενός).

Εν συνεχεία, γίνεται **One-hot encoding** στα features '**Origin/Ethnicity**' και '**Genre**' με την χρήση της **pd.get_dummies** (σε κάθε **set**), τα οποία περνιούνται με τα κατάλληλα **keys** στα **dictionaries** που δημιουργούμε στο επόμενο βήμα. Από το shape[1] ενός εκ των τριών genre_dummies, παίρνουμε τις 20 διαφορετικές τιμές του feature 'Genre' ώστε να το χρησιμοποιήσουμε αργότερα.

Πριν περάσουμε τα '**Title**' και '**Plot**' στα **dictionaries**, τα περνάμε από την παραπάνω συνάρτηση επεξεργασίας κειμένου με την χρήση της **apply()**. 

Δημιουργούμε τα **dictionaries** και ύστερα τα περνάμε με την **from_tensor_slices()** (των **tf.data.Dataset**) στα Dataset object (**raw_train_ds**, **raw_val_ds**, **raw_test_ds**), και τέλος τα επιστρέφουμε. 

In [ ]:
def load_data_wiki(file_path, batch_size=32, p_train=0.65, p_val=0.15):
    
    df = pd.read_csv(url)
    df = df.sample(frac=1, random_state=42)

    n = df.shape[0]
    n_train = int(n * p_train)
    n_val = int(n * p_val)
    n_test = n - n_train - n_val

    df_train, df_val, df_test = df.iloc[:n_train], df.iloc[n_train:n_train+n_val],\
                                                          df.iloc[n_train+n_val:]

    origin_dummies1 = pd.get_dummies(df_train['Origin/Ethnicity'])
    genre_dummies1 = pd.get_dummies(df_train['Genre'])

    origin_dummies2 = pd.get_dummies(df_val['Origin/Ethnicity'])
    genre_dummies2 = pd.get_dummies(df_val['Genre'])

    origin_dummies3 = pd.get_dummies(df_test['Origin/Ethnicity'])
    genre_dummies3 = pd.get_dummies(df_test['Genre'])

    n_labels = genre_dummies1.shape[1]

    df['Title'] = df['Title'].apply(preprocess_text)
    df['Plot'] = df['Plot'].apply(preprocess_text)  
                                                            
    train_data = {'Title': df_train['Title'].values,
                  'Origin': origin_dummies1.values.astype(np.float64),
                  'Plot': df_train['Plot'].values,
                  'Genre': genre_dummies1.values.astype(np.float64)}

    val_data = {'Title': df_val['Title'].values,
                'Origin': origin_dummies2.values.astype(np.float64),
                'Plot': df_val['Plot'].values,
                'Genre': genre_dummies2.values.astype(np.float64)}            

    test_data = {'Title': df_test['Title'].values,
                 'Origin': origin_dummies3.values.astype(np.float64),
                 'Plot': df_test['Plot'].values,
                 'Genre': genre_dummies3.values.astype(np.float64)}

    raw_train_ds = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
    raw_val_ds = tf.data.Dataset.from_tensor_slices(val_data).batch(batch_size)
    raw_test_ds = tf.data.Dataset.from_tensor_slices(test_data).batch(batch_size)

    return raw_train_ds, raw_val_ds, raw_test_ds, n_labels

In [ ]:
raw_train_ds, raw_val_ds, raw_test_ds, n_labels = load_data_wiki(url)

## **Έλεγχος των δεδομένων**
Εδώ θα τσεκάρουμε τον **αριθμό των διαφορετικών Genre**, καθώς και αν τα **Title**, **Genre**, **Origin** και **Plot**, της πρώτης ταινίας του πρώτου batch, έχουν περαστεί σωστά (**Title** και **Plot** επεξεργασμένα σωστά - **Origin** και **Genre** **one-hot encoded**)

In [ ]:
n_labels

In [ ]:
for movie in raw_train_ds.take(1):
  print('First movie, from the first batch:')
  print('Title (lowercase etc.): ', movie['Title'][0].numpy())
  print('Genre (One-hot): ', movie['Genre'][0].numpy())
  print('Origin (One-hot): ', movie['Origin'][0].numpy())
  print('Plot (lowercase etc.): ', movie['Plot'][0].numpy())


## **Πρώτο Vectorization**
Εδώ θα υλοποιήσουμε μία διαδικασία η οποία κάνει vectorize τα **Title** και **Plot**, ώστε να δημιουργηθεί μια δυαδική αναπαράσταση των λέξεων (**bag of words**) με λεξιλόγιο (αριθμό λέξεων) που θα ορίσουμε εμείς.

* Το πρώτο λεξιλόγιο θα έχει **500 λέξεις** (**max_features**). 
* Το **output** θα είναι **binary** (1 αν υπάρχει η λέξη, 0 αν δεν υπάρχει)
* Με το **pad_to_max_tokens**, βάζοντας το **True**, του λέμε να συμπληρώσει με **μηδενικά** έως ότου οι λέξεις να είναι όσο το **max_features** που θα του δώσουμε.

In [ ]:
max_features = 500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

In [ ]:
def vectorize_text(text):
  return vectorize_layer(tf.expand_dims(text, -1))

# **Χρήση του Title**
Για αρχή χρησιμοποιούμε μόνο το **title** ως είσοδο (χωρίς το Plot), και το δίνουμε όπως και το κάνουμε **adapt** στο **vectorize_layer**, ώστε παρακάτω να γίνει **vectorized** πριν δωθεί στα τελικά sets.

In [ ]:
def final_sets(raw_train_ds, raw_val_ds, raw_test_ds):
    train_titles = raw_train_ds.map(lambda x: x['Title'])
    vectorize_layer.adapt(train_titles)
    val_titles = raw_val_ds.map(lambda x: x['Title'])
    vectorize_layer.adapt(val_titles)
    test_titles = raw_test_ds.map(lambda x: x['Title'])
    vectorize_layer.adapt(test_titles)

    train_ds = raw_train_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))
    val_ds = raw_val_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))
    test_ds = raw_test_ds.map(lambda x: (vectorize_text(x['Title']), x['Genre']))

    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = final_sets(raw_train_ds, raw_val_ds, raw_test_ds)

## **Πρώτο μοντέλο** 
Γίνεται η εκπαίδευση του πρώτου μοντέλου με χρήση μόνο του **Title**, για την πρόβλεψη του **Genre**.

**10020 παράμετροι** - **Είσοδος = 500**,  **Έξοδος = 20**

In [ ]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

## **Αποτελέσματα**
Χρησιμοποιήθηκε ως **optimizer** η μέθοδος **Adam**, με **learning rate** **0.001**, και **default** οι υπόλοιπες παράμετροι.

Στις 10 εποχές είχαμε **Accuracy: 0.303**, και **Loss: 0.161**

Στις 20 εποχές είχαμε **Accuracy: 0.316**, και **Loss: 0.158**

**Δεν δοκιμάστηκε** σε παραπάνω εποχές μιας και θα το ξανατρέξουμε για **λεξιλόγιο 10000 λέξεων** ώστε να δούμε αν υπάρχει κάποια διαφορά. 

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

## **Λεξιλόγιο 10000 λέξεων**
Αυξάνουμε τις λέξεις στο **bag of words**, και ξαναδημιουργούμε τα **sets**. Ύστερα εκπαιδεύουμε πάλι το μοντέλο μας και το τρέχουμε με τις **ίδιες παραμέτρους** για **20 εποχές**, για να το συγκρίνουμε με το **πρώτο**.

Πλέον, έχουμε **200020 παραμέτρους**, **Είσοδο=10000**, **Έξοδο=20**.

In [ ]:
max_features = 10000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets(raw_train_ds, raw_val_ds, raw_test_ds)

In [ ]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

## **Αποτελέσματα 2**
Στις **20 εποχές** έχουμε **Accuracy: 0.355** και **Loss: 0.159**.


**Απάντηση στην ερώτηση**:

Τα έχει πάει σίγουρα καλύτερα από το πρώτο, αλλά μεγάλη διαφορά δεν υπάρχει παρ' όλο που αυξήθηκαν οι λέξεις στο λεξιλόγιο, κι αυτό οφείλεται, κατά πάσα πιθανότητα, στο ότι με τον τίτλο μόνο, το μοντέλο δεν μπορεί να πάρει τις απαραίτητες πληροφορίες στην εκπαίδευση του. 

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

# **Χρήση του Plot**
Τώρα με την χρήση της συνάρτησης **final_sets_plot**, θα δημιουργήσουμε νέα sets τα οποία θα έχουν μόνο το **Plot**, ώστε να δοκιμάσουμε να εκπαιδεύσουμε ένα μοντέλο με τα **σενάρια των ταινιών** και να ελέγξουμε τα αποτελέσματα του.

In [ ]:
def final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds):
    train_titles = raw_train_ds.map(lambda x: x['Plot'])
    vectorize_layer.adapt(train_titles)
    val_titles = raw_val_ds.map(lambda x: x['Plot'])
    vectorize_layer.adapt(val_titles)
    test_titles = raw_test_ds.map(lambda x: x['Plot'])
    vectorize_layer.adapt(test_titles)

    train_ds = raw_train_ds.map(lambda x: (vectorize_text(x['Plot']), x['Genre']))
    val_ds = raw_val_ds.map(lambda x: (vectorize_text(x['Plot']), x['Genre']))
    test_ds = raw_test_ds.map(lambda x: (vectorize_text(x['Plot']), x['Genre']))

    return train_ds, val_ds, test_ds

Θα χρησιμοποιήσουμε ένα **λεξιλόγιο 1000 λέξεων**, και όλες τις υπόλοιπες παραμέτρους ίδιες.

In [ ]:
max_features = 1000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

**Παράμετροι=20020**, **Είσοδος=1000**, **Έξοδος=20**

In [ ]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

## **Αποτελέσματα 3**
Έχουμε **καθαρή βελτίωση** από τα προηγούμενα, μιας και πλέον έχει ως είσοδο τα **σενάρια** των ταινιών από τα οποία μπορεί να αντλήσει σαφώς περισσότερες πληροφορίες, κι έτσι με **λεξιλόγιο 1000 λέξεων** τα πήγε πολύ καλύτερα από ότι τα είχε πάει το μοντέλο που εκπαιδεύτηκε στους **τίτλους** των ταινιών με **λεξιλόγιο 10000 λέξεων**.

**Accuracy: 0.454**, **Loss: 0.139**

Παρατηρούμε όμως ότι μετά την **7η εποχή** το **val_loss** μόνο αυξάνεται και γενικά ενώ το μοντέλο πετυχαίνει **accuracy** στο train set τιμές υψηλότερες του **0.57**, δεν γενικεύει αποτελεσματικό, περίπτωση **υπερεκπαίδευσης**. Θα γίνει μία δοκιμή και στις **10 εποχές**.

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

## **Εκπαίδευση με 10 εποχές για αποφυγή υπερεκπαίδευσης**
Πράγματι έχουμε **Accuracy: 0.463**, και **Loss: 0.131**. Υπάρχει μικρή βελτίωση, όχι αξιοσημείωτη.

In [ ]:
max_features = 1000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

In [ ]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
max_features = 30000
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='binary',
    pad_to_max_tokens=True)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

**Παράμετροι=600020**, **Είσοδος=30000**, **Έξοδος=20**

In [ ]:
model = tf.keras.Sequential([layers.Dense(20, activation='sigmoid', input_shape=(max_features,))])
model.summary()

## **Αποτελέσματα 4**
Στις **20 εποχές** έχουμε **Accuracy: 0.502** και **Loss: 0.199**

Στις **10 εποχές** έχουμε **Accuracy: 0.490** και **Loss: 0.290**

Στις **5 εποχές** έχουμε **Accuracy: 0.499** και **Loss: 0.238**

Οπότε στις 20 εποχές (που έχουν τρέξει και τα υπόλοιπα) είχαμε το καλύτερο αποτέλεσμα, το οποίο γενικά ίσως θεωρείται χαμηλό σαν accuracy, αλλά είναι σαφώς καλύτερο από τυχαία επιλογή ανάμεσα σε 20 κλάσεις (όπως είπατε).

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

# **Ενσωματώσεις λέξεων μόνο με χρήση Title**
Έχοντας αλλάξει το output_mode του **vectorize_layer** σε '**int**', και περιορίζοντας τους **τίτλους** σε **250 λέξεις** (έτσι κι αλλιώς είναι μικρότεροι), επιχειρούμε τον τρόπο αναπαράστασης με **word embeddings**.

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_ds, val_ds, test_ds = final_sets(raw_train_ds, raw_val_ds, raw_test_ds)

In [ ]:
for movie in train_ds.take(1):
  print(movie[0])

**Ενσωμάτωση λέξεων** με **16 διαστάσεις**.

In [ ]:
embedding_dim = 16
model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(20)])

model.summary()

## **Αποτελέσματα 5**
Στις **20 εποχές** έχουμε **Accuracy: 0.267** και **Loss: 0.160**, το οποίο είναι πολύ χαμηλό. Όχι ιδιαίτερα χαμηλότερο από την εκπαίδευση με **bag of words** λεξιλογίου ίδιας ποσότητας λέξεων, με **χρήση τίτλου**, όπου είχαμε **Accuracy: 0.35**, αλλά σίγουρα χαμηλότερη. 

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

# **Ενσωματώσεις λέξεων μόνο με χρήση Plot**
Τέλος θα δοκιμάσουμε τα ίδια ακριβώς, με χρήση μόνο **σεναρίου**. 

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_ds, val_ds, test_ds = final_sets_plot(raw_train_ds, raw_val_ds, raw_test_ds)

In [ ]:
embedding_dim = 16
model = tf.keras.Sequential([
  layers.Embedding(max_features, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(20)])

model.summary()

## **Αποτελέσματα 6**
Πλέον, στις **20 εποχές** έχουμε **Accuracy: 0.433** και **Loss: 0.139**, τα οποία είναι πάλι χαμηλότερα των αντίστοιχων αποτελεσμάτων με χρήση **bag of words** αναπαράστασης, **λεξιλογίου 1000 λέξεων**, αλλά εδώ θα παρατηρήσουμε την μεγάλη διαφορά μεταξύ:

**Word embeddings με χρήση τίτλου** και **word embeddings με χρήση σεναρίου**. **Accuracy πρώτου: 0.267**, **Accuracy δεύτερου: 0.433**. Παρατηρούμε πόσο σημαντική είναι η **ποσότητα του text** στα **embeddings**, διαφορά η οποία δεν υπάρχει σε τέτοιο μέγεθος στα αντίστοιχα bag of words αποτελέσματα.

Σίγουρα λοιπόν είχαμε **παρόμοιου επιπέδου αποτελέσματα** στις δύο αναπαραστάσεις, αλλά η πραγματική διαφορά υπήρξε στο μέγεθος βελτίωσης των **word embeddings** δίνοντας τους περισσότερο text (**Plot**). 

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
epochs = 20
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)